# Swinging Door #

Реализация алгоритма Swinging Door на Python.

Литература:
 * [Патент US4669097](https://patents.google.com/patent/US4669097 "patents.google.com");
 * [Swinging Door Trending Compression Algorithm for IoT Environments](https://sol.sbc.org.br/index.php/sbesc_estendido/article/download/8650/8551/#:~:text=The%20Swing%20Door%20Trending%20(SDT,process%20information%20systems%20(PIMs). "sol.sbc.org.br");
 * [Компрессия данных в системах промышленной автоматизации. Алгоритм SwingingDoor](https://habr.com/ru/post/105652/ "habr.com");

Реализация:

In [1]:
def swinging_door(data, deviation=.1, mode=False, step=10):
    current_step = 0
    upper_pivot = lower_pivot = current = (0., 0.)

    sloping_upper_max = sloping_lower_min = 0.

    for i, item in enumerate(data):
        if not i:
            entrance = current = item

            upper_pivot = (entrance[0], entrance[1] + deviation)
            lower_pivot = (entrance[0], entrance[1] - deviation)

            yield entrance

            current_step = 0
            continue

        past, current = current, item

        sloping_upper = (current[1] - upper_pivot[1]) / (current[0] - upper_pivot[0])
        sloping_lower = (current[1] - lower_pivot[1]) / (current[0] - lower_pivot[0])

        if not sloping_upper_max and not sloping_lower_min:
            sloping_upper_max = sloping_upper
            sloping_lower_min = sloping_lower

            current_step += 1
            continue

        if sloping_upper > sloping_upper_max:
            sloping_upper_max = sloping_upper

            if sloping_upper_max > sloping_lower_min:
                entrance = past if mode else ((past[0] + current[0]) / 2, (past[1] + current[1]) / 2 - (deviation / 2))

                yield entrance

                current_step = 0

                upper_pivot = entrance[0], entrance[1] + deviation
                lower_pivot = entrance[0], entrance[1] - deviation

                sloping_upper_max = (current[1] - upper_pivot[1]) / (current[0] - upper_pivot[0])
                sloping_lower_min = (current[1] - lower_pivot[1]) / (current[0] - lower_pivot[0])

        elif sloping_lower < sloping_lower_min:
            sloping_lower_min = sloping_lower

            if sloping_upper_max > sloping_lower_min:
                entrance = past if mode else ((past[0] + current[0]) / 2, (past[1] + current[1]) / 2 - (deviation / 2))

                yield entrance

                current_step = 0

                upper_pivot = entrance[0], entrance[1] + deviation
                lower_pivot = entrance[0], entrance[1] - deviation

                sloping_upper_max = (current[1] - upper_pivot[1]) / (current[0] - upper_pivot[0])
                sloping_lower_min = (current[1] - lower_pivot[1]) / (current[0] - lower_pivot[0])

        if mode and current_step == step:
            entrance = past

            yield entrance

            current_step = 0

            upper_pivot = entrance[0], entrance[1] + deviation
            lower_pivot = entrance[0], entrance[1] - deviation

            sloping_upper_max = (current[1] - upper_pivot[1]) / (current[0] - upper_pivot[0])
            sloping_lower_min = (current[1] - lower_pivot[1]) / (current[0] - lower_pivot[0])

        else:
            current_step += 1

    yield current


Тестирование:

In [2]:
assert list(swinging_door([
    (0., 5.0), (1., 5.5), (2., 4.2),
    (3., 5.8), (4., 5.2), (5., 2.8),
], deviation=1.)) == [(0.0, 5.0), (4.5, 3.5), (5.0, 2.8)]

assert list(swinging_door([
    (0., 5.0), (1., 5.5), (2., 4.2),
    (3., 5.8), (4., 5.2), (5., 6.8),
], deviation=1.)) == [(0.0, 5.0), (4.5, 5.5), (5.0, 6.8)]

assert list(swinging_door([
    (0., 5.0), (1., 5.5), (2., 4.2),
    (3., 5.8), (4., 5.2), (5., 6.8),
], deviation=1., mode=True)) == [(0.0, 5.0), (4.0, 5.2), (5.0, 6.8)]

assert list(swinging_door([
    (0., 5.0), (1., 5.5), (2., 4.2),
    (3., 5.8), (4., 5.2), (5., 6.8),
], deviation=1., mode=True, step=2)) == [(0.0, 5.0), (2.0, 4.2), (5.0, 6.8)]

Проверка:

In [3]:
from matplotlib.pyplot import plot, show
from ipywidgets import FloatSlider, Checkbox, IntSlider, interact

my_data = [
    (0., 5.0), (1., 5.5), (2., 4.2),
    (3., 5.8), (4., 5.2), (5., 6.8),
]

def re_show(deviation, step, modernized):
    o_data = list()
    o_index = list()
    
    for x, y in my_data:
        o_data.append(y)
        o_index.append(x)
    
    n_data = list()
    n_index = list()

    for x, y in swinging_door(my_data, deviation, mode=modernized, step=step):
        print(x, y)
        n_data.append(y)
        n_index.append(x)

    plot(o_index, o_data)
    plot(n_index, n_data)

    show()
    
interact(
    re_show,
    deviation=FloatSlider(min=.01, max=10, step=.01, value=1),
    step=IntSlider(min=1, max=100, step=1, value=2),
    modernized=Checkbox(value=False, description="Modernized", disabled=False)
)

interactive(children=(FloatSlider(value=1.0, description='deviation', max=10.0, min=0.01, step=0.01), IntSlide…

<function __main__.re_show(deviation, step, modernized)>

---

## Применение

Постройка графика из сырых данных:

In [1]:
from datetime import datetime

from pandas import read_csv
from matplotlib.pyplot import figure, show
from ipywidgets import FloatSlider, Checkbox, IntSlider, interact

from swinging_door import swinging_door

data = [
    (datetime.strptime(date, "%Y-%m-%d").timestamp(), value)
    for date, value in read_csv(
        "https://datahub.io/core/oil-prices/r/wti-daily.csv"
    ).values.tolist()
]

def re_show(deviation, step, modernized):
    x = list()
    y = list()
    
    for k, v in data:
        x.append(datetime.fromtimestamp(k))
        y.append(v)

    s_x = list()
    s_y = list()

    s_data = list(
        swinging_door(data, deviation, mode=modernized, step=step)
    )

    for k, v in s_data:
        s_x.append(datetime.fromtimestamp(k))
        s_y.append(v)

    fig = figure()
    fig.set_dpi(100)
    fig.suptitle("The operation of the Swinging Door algorithm.")

    gs = fig.add_gridspec(2, hspace=.3)

    (ax_orig, ax_sd) = fig.add_gridspec(2, hspace=.3).subplots(sharex=True, sharey=True)

    ax_orig.set_title("Original data")
    ax_orig.plot(x, y, "tab:blue", label=f"count: {len(data)}")
    ax_orig.legend(loc="upper left")

    ax_sd.set_title(f"Compressed data use deviation {deviation}")
    ax_sd.plot(s_x, s_y, "tab:orange", label=f"count: {len(s_data)}")
    ax_sd.legend(loc="upper left")

    show()
    
interact(
    re_show,
    deviation=FloatSlider(min=.01, max=10, step=.01, value=1),
    step=IntSlider(min=1, max=100, step=1, value=10),
    modernized=Checkbox(value=False, description="Modernized", disabled=False)
)

interactive(children=(FloatSlider(value=1.0, description='deviation', max=10.0, min=0.01, step=0.01), IntSlide…

<function __main__.re_show(deviation, step, modernized)>